In [7]:
import os
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

In [44]:
DIR = os.path.dirname(os.path.abspath(__name__))
DB_PATH = os.path.join(DIR, 'db/')
DB_PATH

'/home/sanchez/VScode_consolidated/task1/db/'

In [45]:
# Connect to the running ChromaDB instance
client = chromadb.PersistentClient(path=DB_PATH)

In [50]:
# Create or get a collection
collection = client.create_collection("pdf_name")

In [51]:
# Add data to the collection
documents = ["This is a beach", "The forest is green", "Urban areas are crowded"]
metadatas = [{"category": "beach"}, {"category": "forest"}, {"category": "urban"}]
ids = ["1", "2", "3"]

collection.add(documents=documents, metadatas=metadatas, ids=ids)

In [52]:
# Query the collection
results = collection.query(
    query_texts=["What are forests like?"],
    n_results=3  # Number of results to return
)

# Print the results
print(results)

{'ids': [['2', '1', '3']], 'distances': [[0.8161929381010665, 1.6312474679542952, 1.6889317385852014]], 'metadatas': [[{'category': 'forest'}, {'category': 'beach'}, {'category': 'urban'}]], 'embeddings': None, 'documents': [['The forest is green', 'This is a beach', 'Urban areas are crowded']], 'uris': None, 'data': None, 'included': ['metadatas', 'documents', 'distances']}


#################SentenceTransformer##############

In [53]:
from sentence_transformers import SentenceTransformer

/home/sanchez/VScode_consolidated/task1/venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [54]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [60]:
DOG_embeddings = model.encode(["DOG"])
dog_embeddings = model.encode(["dog"])
(DOG_embeddings == dog_embeddings).all()

np.True_

In [62]:
good_embeddings = model.encode(["good"])
better_embeddings = model.encode(["better"])
best_embeddings = model.encode(["best"])
print(
  (good_embeddings == better_embeddings).all(),
  (good_embeddings == best_embeddings).all(),
  (better_embeddings == best_embeddings).all()
)

False False False


In [64]:
very_good_embeddings = model.encode(["very good"])
better_embeddings_embeddings = model.encode(["better"])
print((very_good_embeddings == better_embeddings_embeddings).all())

False


In [65]:
#####################Add Langchain#####################

In [79]:
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

In [80]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [81]:
vector_store = Chroma(
    collection_name="documnet_collection",
    embedding_function=embeddings,
    persist_directory="db/",
)

In [82]:
# Add documents
from uuid import uuid4
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
    id=1,
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
    id=2,
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
    id=3,
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
    id=4,
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
    id=5,
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
    id=6,
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
    id=7,
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
    id=8,
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
    id=9,
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
    id=10,
)


In [83]:
documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]

uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['e7d29e68-229d-40f8-87d5-1dbf8bd2caf2',
 'b0cec3a9-61f6-4716-8727-0e0cf30dfa00',
 '20fdbf09-fb23-4eb5-827d-af5bec0e3cb8',
 'c8eb90f0-805b-4dee-8bee-602beb4cb5de',
 '6b7ec753-c9fb-455e-a006-ec1cff479d09',
 'aebb41cf-a7f4-40fe-953c-74b0e8f3fcf3',
 '755e632f-f0b3-4f74-be97-0693074cfe6f',
 '598fc993-8436-4d5c-98d0-ffb00bb16935',
 '02fa4dd1-ee69-49a3-9c06-41281da5855a',
 'd4620ab6-0d27-4f80-9bfc-afc3ae3f5836']

########################Add PDF reader################

In [138]:
from helpers import *

async def upload_pdf(pdf_file:str):
  ids = list()
  id_calibration = 0
  async for metadata, chunks in load_pdf(pdf_file):
    for id, chunk in enumerate(chunks):
      chunk.metadata.update(metadata)
      ids.append(id_calibration+id)

    # await upload_to_supabase(chunks,ids)
    print(chunks,ids)
    break
    id_calibration = ids[-1]
    ids = list()

In [139]:
await upload_pdf("WekaManual_13to15.pdf")

page: 1


NameError: name 'split_document' is not defined